In [23]:
#### Partying Decision Example ####
import random #Adding a utility function

def argmax(gen):
    """gen is a generator of (element,value) pairs, where value is a real.
    argmax returns an element with maximal value.
    If there are multiple elements with the max value, one is returned at random.
    """
    maxv = float('-Infinity')       # negative infinity
    maxvals = []      # list of maximal elements
    for (e,v) in gen:
        if v>maxv:
            maxvals,maxv = [e], v
        elif v==maxv:
            maxvals.append(e)
    return random.choice(maxvals)

# Try:
# argmax(enumerate([1,6,3,77,3,55,23]))

def flip(prob):
    """return true with probability prob"""
    return random.random() < prob

def dict_union(d1,d2):
    """returns a dictionary that contains the keys of d1 and d2.
    The value for each key that is in d2 is the value from d2,
    otherwise it is the value from d1.
    This does not have side effects.
    """
    d = dict(d1)    # copy d1
    d.update(d2)
    return d

def test():
    """Test part of utilities"""
    assert argmax(enumerate([1,6,55,3,55,23])) in [2,4]
    assert dict_union({1:4, 2:5, 3:4},{5:7, 2:9}) == {1:4, 2:9, 3:4, 5:7} 
    print("Passed unit test in utilities")

if __name__ == "__main__":
    test()


class MDP(object):#A clas for markovian decision processes.See the transition model
    def __init__(self, states, actions, trans, reward, discount):
        """states is a list or tuple of states.
        actions is a list or tuple of actions
        trans[s][a][s'] represents P(s'|a,s)
        reward[s][a] gives the expected reward of doing a in state s
        discount is a real in the range [0,1]
        """
        self.states = states
        self.actions = actions
        self.trans = trans
        self.reward = reward
        self.discount = discount
        self.v0 = [0 for s in states]  # initial value function
        
    def vi1(self,v):
         """carry out one iteration of value iteration and 
         returns a value function (a list of a value for each state).
         v is the previous value function.
         """
         return [max([self.reward[s][a]+self.discount*product(self.trans[s][a],v)
                      for a in range(len(self.actions))])
                 for s in range(len(self.states))]

    def vi(self,v0,n):
        """carries out n iterations of value iteration starting with value v0.

        Returns a value function
        """
        val = self.v0
        for i in range(n):
           val= self.vi1(val)
        return val

    def policy(self,v): 
         """returns an optimal policy assuming the next value function is v
            v is a list of values for each state
            returns a list of the indexes of optimal actions for each state
         """
         return [argmax(enumerate([self.reward[s][a]+self.discount*product(self.trans[s][a],v)
                                   for a in range(len(self.actions))]))
                 for s in range(len(self.states))]

    def q(self,v): 
        """returns the one-step-lookahead q-value assuming the next value function is v
        v is a list of values for each state
        returns a list of q values for each state. so that q[s][a] represents Q(s,a)
        """
        return [[self.reward[s][a]+self.discount*product(self.trans[s][a],v)
                  for a in range(len(self.actions))]
                 for s in range(len(self.states))]
    
def product(l1,l2):
    """returns the dot product of l1 and l2"""
    return sum([i1*i2 for (i1,i2) in zip(l1,l2)])



# States: Healthy Sick
# Actions: Relax Party

# trans[s][a][s'] gives P(s'|a,s)
#           Relax        Party
trans2 = (((0.95,0.05), (0.7, 0.3)),   # Healthy
          ((0.5,0.5),   (0.1, 0.9))    # Sick
        )

# reward[s][a] gives the expected reward of doing a in state s.
reward2 = ((7,10),(0,2))

healthy2 = MDP(['Healthy','Sick'], ['Relax','Party'], trans2, reward2, discount=0.8) 

## Tiny Game from Example 11.7 and Figure 11.8 of Poole and Mackworth, 2010 #

#See a transition model that the MDP should utilize
# actions        up                right          upC             left
transt = (((0.1,0.1,0.8,0,0,0), (0,1,0,0,0,0), (0,0,1,0,0,0), (1,0,0,0,0,0)), #s0
         ((0.1,0.1,0,0.8,0,0), (0,1,0,0,0,0), (0,0,0,1,0,0), (1,0,0,0,0,0)),  #s1
         ((0,0,0.1,0.1,0.8,0), (0,0,0,1,0,0), (0,0,0,0,1,0), (0,0,1,0,0,0)),  #s2
         ((0,0,0.1,0.1,0,0.8), (0,0,0,1,0,0), (0,0,0,0,0,1), (0,0,1,0,0,0)),  #s3
         ((0.1,0,0,0,0.8,0.1), (0,0,0,0,0,1), (0,0,0,0,1,0), (1,0,0,0,0,0)),  #s4
         ((0,0,0,0,0.1,0.9),   (0,0,0,0,0,1), (0,0,0,0,0,1), (0,0,0,0,1,0)) ) #s5

# actions     up  rt  upC  left
rewardt = ((-0.1,  0,  -1,   -1),   #s0
           (-0.1, -1,  -2,    0),   #s1
           (-10,   0,  -1, -100),   #s2
           (-0.1, -1,  -1,    0),   #s3
           (-1,    0,  -2,   10),   #s4
           (-1,   -1,  -2,    0))   #s5
#Call the MDP process
mdpt = MDP(['s0','s1','s2','s3','s4','s5'],   # states
           ['up', 'right', 'upC', 'left'],    # actions
           transt, rewardt, discount=0.9)

def trace(mdp,numiter):
    print("Q values are shown as",[[st+"_"+ac for ac in mdp.actions] for st in mdp.states])
    print("One step lookahead Q-values:")
    print(mdp.q(mdp.v0))
    print("Values are for the states:", mdp.states)
    print("One step lookahead values:")
    print(mdp.vi(mdp.v0,1))
    print("Two step lookahead Q-values:")
    print(mdp.q(mdp.vi(mdp.v0,1)))
    print("Two step lookahead values:")
    print(mdp.vi(mdp.v0,2))
    vfin = mdp.vi(mdp.v0,numiter)
    print("After",numiter,"iterations, values:")
    print(vfin)
    print("After",numiter,"iterations, Q-values:")
    print(mdp.q(vfin))
    print("After",numiter,"iterations, Policy:", 
           [st+"->"+mdp.actions[act] for (st,act) in zip(mdp.states ,mdp.policy(vfin))])

# Try the following:
# trace(healthy2,10)



Passed unit test in utilities


In [24]:
trace(healthy2,10)

Q values are shown as [['Healthy_Relax', 'Healthy_Party'], ['Sick_Relax', 'Sick_Party']]
One step lookahead Q-values:
[[7.0, 10.0], [0.0, 2.0]]
Values are for the states: ['Healthy', 'Sick']
One step lookahead values:
[10.0, 2.0]
Two step lookahead Q-values:
[[14.68, 16.08], [4.800000000000001, 4.24]]
Two step lookahead values:
[16.08, 4.800000000000001]
After 10 iterations, values:
[32.45950570966077, 20.554744073232058]
After 10 iterations, Q-values:
[[32.49141410227146, 33.11046177498572], [21.205699913157133, 19.396176189499943]]
After 10 iterations, Policy: ['Healthy->Party', 'Sick->Relax']
